In [ ]:
import sys
import os

# Path to the directory containing GABC.py
custom_path = './../lib/'
# Add the custom path to the system path
sys.path.append(custom_path)
sys.path.append("./../")
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
import pyminion

In [ ]:
function_name_to_function = {
    "sphere": sphere,
    "rosenbrock": rosenbrock,
    "rastrigin": rastrigin,
    "drop_wave":  drop_wave, 
    "griewank": griewank,
    "ackley": ackley,
    "zakharov": zakharov,
    "goldstein_price" : goldstein_price,
    "michalewicz": michalewicz,
    "easom" : easom,
    "levy": levy,
    "dixon_price": dixon_price,
    "bent_cigar": bent_cigar,
    "discus": discus,
    "weierstrass": weierstrass,
    "happy_cat": happy_cat,
    "hgbat": hgbat,
    "hcf": hcf,
    "grie_rosen": grie_rosen,
    "escaffer6": escaffer6,
    "hybrid_composition1": hybrid_composition1,
    "hybrid_composition2": hybrid_composition2,
    "hybrid_composition3": hybrid_composition3,
    "step": step,
    "quartic": quartic,
    "schaffer2": schaffer2,
    "brown": brown,
    "exponential": exponential,
    "styblinski_tang": styblinski_tang,
    "sum_squares": sum_squares
}

# Map function implementations to function names
function_to_function_name = {v: k for k, v in function_name_to_function.items()}

# Function evaluation counter
class FunctionEvaluator:
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += 1
        ret = self.func(np.array([X]))[0]
        #print(X.shape, ret.shape)
        return ret
    
class VectorizedEvaluator : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += X.shape[0]
        ret = self.func(np.array(X))
        return ret
    
class PyminionFunc : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X, data=None):
        X= np.array(X)
        self.n_calls += X.shape[0]
        return  self.func(X)
    
class VectorizedEvaluatorDE : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        X = np.array(X).T
        self.n_calls += X.shape[0]
        return self.func(X)


In [ ]:
results = []
def test_optimization(func, bounds, dimension, func_name, Nmaxeval):
    result = {}
    result['Function'] = func_name
    result['Dimensions'] = dimension
    # Initialize bounds
    bounds_list = [bounds] * dimension

    # Create wrapped function evaluator
    evaluator = FunctionEvaluator(func)
    vecEvaluator = VectorizedEvaluator(func)
    vecEvaluatorDE = VectorizedEvaluatorDE(func)
    pyminionFunc = PyminionFunc(func)

      #SHADE 
    vecEvaluator.n_calls =0
    shade = M_LSHADE_AMR(func=vecEvaluator, bounds=bounds_list, maxevals=Nmaxeval, population_size=30,  relTol=0.0,
                  strategy= 'current_to_pbest1bin',  seed=None, memorySize=30, minPopSize=10)
    res = shade.optimize()
    result['M_LSHADE_AMR Value'] = res.fun
    vecEvaluator.n_calls =0

     # Lshade Optimization
    start = time.time()
    shade_ = pyminion.M_LSHADE_AMR (pyminionFunc, bounds_list, data=None, x0=None, population_size=30, maxevals=Nmaxeval, 
                 strategy="current_to_pbest1bin", relTol=0.0, minPopSize=10, memeorySize=30, callback=None, boundStrategy="reflect-random", seed=None)
    jde_result = shade_.optimize()
    #print("TIME c++ : ", time.time()-start)
    result['C++ M_LSHADE_AMR Value'] = jde_result.fun
    vecEvaluator.n_calls =0


    # LJADE Optimization
    start = time.time()
    jde_optimizer = M_LJADE_AMR(func=vecEvaluator, bounds=bounds_list, maxevals=Nmaxeval, population_size=30, relTol=0.0,
                         strategy= 'current_to_pbest1bin',  minPopSize=10, c=0.5, seed=None)
    jde_result = jde_optimizer.optimize()
    #print("TIME cython : ", time.time()-start)
    result['M_LJADE_AMR Value'] = jde_result.fun
    vecEvaluator.n_calls =0

    # LJADE Optimization
    start = time.time()
    jde_ = pyminion.M_LJADE_AMR (pyminionFunc, bounds_list, data=None, x0=None, population_size=30, maxevals=Nmaxeval, 
                 strategy="current_to_pbest1bin", relTol=0.0, minPopSize=10, c=0.5, callback=None, boundStrategy="reflect-random", seed=None)
    jde_result = jde_.optimize()
    #print("TIME c++ : ", time.time()-start)
    result['C++ M_LJADE_AMR Value'] = jde_result.fun
    vecEvaluator.n_calls =0

    # Global best artifical bee colony opimization
    vecEvaluator.n_calls=0
    abc_optimizer = GABC(vecEvaluator, bounds_list, population_size=30, maxevals = Nmaxeval,  influence_global_best=2.0,
                 mutation_prob=0.1, mutation_rate=0.5, verbose=False, patience=1000)
    abc_optimizer.optimize()
    result['GABC Value'] = abc_optimizer.best_value
    vecEvaluator.n_calls =0

    # Differential Evolution (DE)
    vecEvaluatorDE.n_calls = 0
    de_result = differential_evolution(vecEvaluatorDE, bounds_list,popsize=max(int(30/dimension), 1), strategy='best1exp',
                                         maxiter=int(Nmaxeval/(2*dimension)), vectorized=True, updating="deferred", disp=False,polish=False)
    result['Scipy DE Value'] = de_result.fun


    results.append(result)
    print(result)
    

    if False:
      plt.figure(figsize=(4,3))
      plt.plot(jde_.muCR, label ="C++ CR")
      plt.plot(jde_.muF, label="F")
      plt.legend()
      plt.show()
     


      plt.figure(figsize=(4,3))
      plt.plot(jde_optimizer.muCR, label="MuCR")
      plt.plot(jde_optimizer.muF, label="MuF")
      plt.legend()
      plt.show()


      plt.figure(figsize=(4,3))
      plt.plot(shade_.muCR, label ="C++ CR")
      plt.plot(shade_.muF, label="F")
      plt.legend()
      plt.show()
     


      plt.figure(figsize=(4,3))
      plt.plot(shade.muCR, label="MuCR")
      plt.plot(shade.muF, label="MuF")
      plt.legend()
      plt.show()

      #plt.plot(np.array(jde_optimizer.history)[:, 1], label="Fitness JDE")
      #plt.plot(np.array(shade.history)[:,1], label="Fitness SHADE")
      #plt.plot([h.fun for h in jde_.history], label="Fitness JADE C++")
      #plt.plot(fitnesses_, label="Fitness LSHADE", color="cyan", linestyle="dashed")
      #plt.legend()
      #plt.yscale("log")
      #plt.show() 
      
Nmaxeval = 10000
dimensions = [ 20]
test = function_name_to_function #{"Any" : rastrigin}
#test = {"Any" : hybrid_composition1}
for dim in dimensions:
    i=0
    for funcname, func in test.items() : #function_name_to_function.items():
        test_optimization(func, (-10, 10), dim, funcname, Nmaxeval)
        i=i+1 
        if i>15: break 

# Results output
import pandas as pd
pd.set_option('display.float_format', '{:.12f}'.format)
results_df = pd.DataFrame(results)
results_df

In [ ]:
results = []
def test_optimization(func, bounds, dimension, func_name, Nmaxeval):
    result = {}
    result['Function'] = func_name
    result['Dimensions'] = dimension
    # Initialize bounds
    bounds_list = [bounds] * dimension

    # Create wrapped function evaluator
    evaluator = FunctionEvaluator(func)
    vecEvaluator = VectorizedEvaluator(func)
    vecEvaluatorDE = VectorizedEvaluatorDE(func)
    pyminionFunc = PyminionFunc(func)

      #SHADE 
    vecEvaluator.n_calls =0
    shade = M_LSHADE_AMR(func=vecEvaluator, bounds=bounds_list, maxevals=Nmaxeval, population_size=30,  relTol=0.0,
                  strategy= 'current_to_pbest1bin',  seed=None, memorySize=30, minPopSize=10)
    res = shade.optimize()
    result['M_LSHADE_AMR Value'] = res.fun
    vecEvaluator.n_calls =0


    # LJADE Optimization
    start = time.time()
    jde_optimizer = M_LJADE_AMR(func=vecEvaluator, bounds=bounds_list, maxevals=Nmaxeval, population_size=30, relTol=0.0,
                         strategy= 'current_to_pbest1bin',  minPopSize=10, c=0.5, seed=None)
    jde_result = jde_optimizer.optimize()
    print("TIME cython : ", time.time()-start)
    result['M_LJADE_AMR Value'] = jde_result.fun
    vecEvaluator.n_calls =0

    # LJADE Optimization
    start = time.time()
    jde_ = pyminion.M_LJADE_AMR (pyminionFunc, bounds_list, data=None, x0=None, population_size=30, maxevals=Nmaxeval, 
                 strategy="current_to_pbest1bin", relTol=0.0, minPopSize=10, c=0.5, callback=None, seed=None)
    jde_result = jde_.optimize()
    print("TIME c++ : ", time.time()-start)
    result['C++ M_LJADE_AMR Value'] = jde_result.fun
    vecEvaluator.n_calls =0

    # Global best artifical bee colony opimization
    vecEvaluator.n_calls=0
    abc_optimizer = GABC(vecEvaluator, bounds_list, population_size=30, maxevals = Nmaxeval,  influence_global_best=2.0,
                 mutation_prob=0.1, mutation_rate=0.5, verbose=False, patience=1000)
    abc_optimizer.optimize()
    result['GABC Value'] = abc_optimizer.best_value
    vecEvaluator.n_calls =0

    # Differential Evolution (DE)
    vecEvaluatorDE.n_calls = 0
    de_result = differential_evolution(vecEvaluatorDE, bounds_list,popsize=max(int(30/dimension), 1), strategy='best1exp',
                                         maxiter=int(Nmaxeval/(2*dimension)), vectorized=True, updating="deferred", disp=False,polish=False)
    result['Scipy DE Value'] = de_result.fun


    results.append(result)
    print(result)
    

    if False:
      plt.figure(figsize=(4,3))
      plt.plot(jde_.muCR, label ="C++ CR")
      plt.plot(jde_.muF, label="F")
      plt.legend()
      plt.show()
     


      plt.figure(figsize=(4,3))
      plt.plot(jde_optimizer.muCR, label="MuCR")
      plt.plot(jde_optimizer.muF, label="MuF")
      plt.legend()
      plt.show()

      #plt.plot(np.array(jde_optimizer.history)[:, 1], label="Fitness JDE")
      #plt.plot(np.array(shade.history)[:,1], label="Fitness SHADE")
      #plt.plot([h.fun for h in jde_.history], label="Fitness JADE C++")
      #plt.plot(fitnesses_, label="Fitness LSHADE", color="cyan", linestyle="dashed")
      #plt.legend()
      #plt.yscale("log")
      #plt.show() 
      
Nmaxeval = 10000
dimensions = [ 30]
test = function_name_to_function #{"Any" : rastrigin}
for dim in dimensions:
    for funcname, func in test.items() : #function_name_to_function.items():
        test_optimization(func, (-10, 10), dim, funcname, Nmaxeval)

# Results output
import pandas as pd
pd.set_option('display.float_format', '{:.3f}'.format)
results_df = pd.DataFrame(results)
results_df

In [ ]:
np.random.uniform(0.1, 0.9)

In [ ]:
dim = 20
bounds_list = [(-1., 1.)]*dim
func= schwefel
shade = M_LSHADE_AMR(func=func, bounds=bounds_list, maxevals=100, population_size=30,  relTol=0.0,
                strategy= 'current_to_pbest1bin',  seed=None, memorySize=30, minPopSize=10)
res = shade.optimize()

plt.figure(figsize=(4,3))
plt.plot(shade.stdCR, label="STDCR")
plt.plot(shade.stdF, label="STDF")
plt.legend()
plt.show() 



In [ ]:
filtered_attributes_and_methods = [attr for attr in dir(shade) if not attr.startswith('__')]

# Print the names and values of the filtered attributes and methods
for attr in filtered_attributes_and_methods:
    attr_value = getattr(shade, attr)
    print(f"Name: {attr}, Value: {attr_value}")